In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

특정 주제에 대한 4지선다형 퀴즈를 생성하는 과정을 구현합니다.

`Quiz` 클래스는 퀴즈의 질문, 난이도, 그리고 네 개의 선택지를 정의합니다.

`ChatOpenAI` 인스턴스는 GPT-4 모델을 사용하여 자연어 처리를 수행하고, `ChatPromptTemplate`는 퀴즈 생성을 위한 대화형 프롬프트를 정의합니다.

마지막으로, `create_structured_output_runnable` 함수는 이러한 구성 요소를 결합하여 주어진 주제(`"ADSP(데이터 분석 준전문가) 자격 시험"`)에 대한 퀴즈를 생성하는 실행 가능한 체인을 만듭니다.


In [5]:
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List


class Quiz(BaseModel):
    """4지선다형 퀴즈의 정보를 추출합니다"""

    question: str = Field(..., description="퀴즈의 질문")
    level: str = Field(
        ..., description="퀴즈의 난이도를 나타냅니다. (쉬움, 보통, 어려움)"
    )
    options: List[str] = Field(..., description="퀴즈의 4개의 선택지 입니다.")


llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.1)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a world-famous quizzer and generates quizzes in structured formats.",
        ),
        (
            "human",
            "TOPIC 에 제시된 내용과 관련한 4지선다형 퀴즈를 출제해 주세요. 만약, 실제 출제된 기출문제가 있다면 비슷한 문제를 만들어 출제하세요. 단, 문제에 TOPIC 에 대한 내용이나 정보는 포함하지 마세요. \nTOPIC:\n{topic}",
        ),
        ("human", "Tip: Make sure to answer in the correct format"),
    ]
)
chain = create_structured_output_runnable(Quiz, llm, prompt)
generated_quiz = chain.invoke({"topic": "ADSP(데이터 분석 준전문가) 자격 시험"})

In [6]:
generated_quiz

Quiz(question='다음 중 데이터 분석 프로젝트에서 데이터 전처리 과정에 포함되지 않는 것은 무엇인가?', level='보통', options=['결측치 처리', '이상치 제거', '데이터 정규화', '모델 평가'])

이 함수는 `generated_quiz` 객체에서 질문(`question`)과 난이도(`level`)를 출력한 후, 해당 퀴즈의 선택지(`options`)를 순회하며 인덱스와 함께 출력합니다. `enumerate` 함수를 사용하여 각 선택지의 인덱스를 가져오고, 이를 사용자에게 보여주기 쉬운 형태로 출력합니다. 이 과정은 퀴즈의 질문과 선택지를 사용자에게 제시하는 데 사용됩니다.


In [9]:
print(f"{generated_quiz.question} (난이도: {generated_quiz.level})\n")
for i, opt in enumerate(generated_quiz.options):
    print(f"{i+1}) {opt}")

다음 중 데이터 분석 프로젝트에서 데이터 전처리 과정에 포함되지 않는 것은 무엇인가? (난이도: 보통)

1) 결측치 처리
2) 이상치 제거
3) 데이터 정규화
4) 모델 평가
